In [ ]:
import requests 
from bs4 import BeautifulSoup 


def addUrls(url, listUrl): 
    #open with GET method 
    resp=requests.get(url)         
    if resp.status_code==200: 
        print("Successfully opened the web page") 
        listUrl.append(url)
    else:
        print("Did not work")
    soup=BeautifulSoup(resp.text,'html.parser')
    l=soup.find("a",{"class":"_2glka"})   #_2glka has to be adjusted based on website  
    if (l.get('href') == ""):
        print('Done adding')
    else:                
        addUrls(l.get('href'), listUrl) 





In [ ]:
url='https://www.apartmentguide.com/apartments/Ohio/Dayton/?WT.mc_id=21000&gclid=Cj0KCQjws5HlBRDIARIsAOomqA1pP7zIojO_5sRmhgPWEX1pbm_qIdlzgtOgyk2fIQFgbeB5jP695hoaAlGJEALw_wcB'
listUrl = []
addUrls(url, listUrl)

In [ ]:
print(listUrl)

In [ ]:
### Some apartments do not have ratings, in this case, NA


def searchApartmentProperties(url, f):
    global oneTime
    resp=requests.get(url)         
    if resp.status_code==200: 
        print("Successfully opened the web page") 
    else:
        print("Did not work")
    soup=BeautifulSoup(resp.text,'html.parser')
    l2=soup.findAll("div",{"data-tid":"listing-info"})
    if(oneTime):
        for prop in l2[1].findAll('div'):
            if(prop.get('data-tid')):
                f.write(prop.get('data-tid')+'#')
        oneTime = False
        f.write('\n')
    index = 0
    done = False
    for listing in l2:    
        for i in listing.findAll('div'):
            #ulaText column is missing or ratings column is missing
            if (index == 4 and i.get('data-tid')=='phone'):
                f.write('NA'+'#'+'NA'+'#')
            if ((index == 5 and done == False) and i.get('data-tid')=='phone'):
                f.write('NA'+'#')                  
            if (index == 4 and i.get('data-tid')=='ratings'):
                f.write('NA'+'#')        
                done = True
     
            
            if(i.get('data-tid')):
                print(i.get('data-tid'))
                print(i.text)
                f.write(i.text+'#')
                index = index + 1
        done = False
        index = 0
        print()
        f.write('\n')

In [ ]:
i = 1
f = open("test.txt", "a")
oneTime = True
for url in listUrl:
    print('page %.f'%i)  
    searchApartmentProperties(url, f)   
    i = i + 1
f.close()

In [ ]:
def deleteChar(inputFile, outputFile, myChar):
    with open(inputFile, "rt") as fin:
        with open(outputFile, "wt") as fout:
            for line in fin:
                fout.write(line.replace(myChar, ''))

In [ ]:
import os
deleteChar('test.txt', 'subList.txt', '(')
deleteChar('subList.txt', 'finalList.txt', ')')
os.remove("subList.txt")

In [ ]:
import csv

with open('finalList.txt', 'r') as infile, open('final.csv', 'w') as outfile:
    stripped = (line.strip() for line in infile)
    lines = (line.split("#") for line in stripped if line)
    writer = csv.writer(outfile)
    writer.writerows(lines)